In [37]:
# import libraries
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tqdm import tqdm
from pycocotools import mask as maskUtils
# interact
from ipywidgets import interact

In [38]:
VIDEO_PATH = '../sources/KIMPO_230808_4.MOV'
LABEL_PATH = '../runs/predict-seg/exp/labels'

# Exists check
if not os.path.exists(VIDEO_PATH):
    print('Video file not exists')
    sys.exit()
if not os.path.exists(LABEL_PATH):
    print('Label file not exists')
    sys.exit()

In [39]:
def polygons_to_rle(polygons, height, width):
    # Create an empty mask
    mask_image = np.zeros((height, width), dtype=np.uint8)

    # Draw the polygons on the mask
    for polygon in polygons:
        # Reshape the polygon to the required format for cv2.fillPoly
        np_polygon = np.array(polygon).reshape((-1, 2))
        cv2.fillPoly(mask_image, [np_polygon], 1)

    # Create RLE from the mask
    rle = maskUtils.encode(np.asfortranarray(mask_image))
    return rle

In [40]:
cap = cv2.VideoCapture(VIDEO_PATH)
max_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
cap.release()

In [35]:
@interact(frame_id=(1, max_frame))
def print_rle(frame_id):
    # cap = cv2.VideoCapture(VIDEO_PATH)
    # cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id-1)
    # ret, frame = cap.read()
    # if not ret:
    #     print("프레임을 가져오는데 실패했습니다.")
    #     cap.release()
    #     return

    file_name = os.path.splitext(os.path.basename(VIDEO_PATH))[0]
    label_file = os.path.join(LABEL_PATH, file_name + '_' + str(frame_id) + '.txt')

    # 레이블 파일 읽기 및 포인트 조정
    if os.path.exists(label_file):
        with open(label_file, 'r') as f:
            data = f.read().strip()

        for line in data.split('\n'):
            parts = line.split()
            label = int(parts[0])
            # points: N x 2 배열
            points = np.array(parts[1:], dtype=float).reshape(-1, 2)
            points[:, 0] *= frame_width  # X 좌표 조정
            points[:, 1] *= frame_height  # Y 좌표 조정
            # put random color
            # color = np.random.randint(0, 255, 3).tolist()
            # cv2.polylines(frame, [points.astype(np.int32)], True, color, 7)
            
            polygons =[]
            for point in points:
                # int casting
                polygons.append([int(point[0]), int(point[1])])
                
            # RLE 변환
            rle = polygons_to_rle(polygons, frame_height, frame_width)
            # rle의 counts 크기 확인
            print('rle counts size: ', len(rle['counts']))
            
    # 프레임 표시
    # plt.figure(figsize=(14, 10))
    # plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    # plt.axis('off')
    # plt.show()
    # cap.release()

interactive(children=(IntSlider(value=612, description='frame_id', max=1224, min=1), Output()), _dom_classes=(…